In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import subprocess
import shlex
# My module
from py_analog import gen_rand
from fileinput import close

print("START: Incident at Pristina airport;\nWe Whant var!")
#https://www.youtube.com/watch?v=ENyxseq59YQ&ab_channel=0STrack

START: Incident at Pristina airport;
We Whant var!


### Random sets generation

In next three cells we are going:
*  Compile C++ code using comp.sh script
*  Make array of sets with random integers
*  Run binary using sets as arguments and save amount of duplicates in <i>'temp.txt'</i>

Setup random generator:
*  <b>start</b> : number of elements for the shortest first set (min>=3)
*  <b>stop</b> : number of elements for the longest last set (max<=1001). Limited with number of arguments of binary file
*  <b>step</b> : increment from <i>start</i> to <i>stop</i>

In [2]:
!make resbt;

start = 3
stop  = 1001
step  = 1

rm -f data/size_dup.txt bin/st_bt
echo "Len Dup" >> data/size_dup.txt;
c++ cpp_src/st_bt.cpp -Wall -Werror -Wextra -std=c++20 -o bin/st_bt
In file included from cpp_src/st_bt.cpp:3:
In file included from cpp_src/../nbt/ser_head/../nbt.h:6:
In file included from cpp_src/../nbt/tree_head/tree_collect.h:4:
cpp_src/../nbt/tree_head/cl_bt.hpp:9:25: error: private field 'data' is not used [-Werror,-Wunused-private-field]
            char*       data;
                        ^
cpp_src/../nbt/tree_head/cl_bt.hpp:10:22: error: private field 'left' is not used [-Werror,-Wunused-private-field]
            Node*    left;
                     ^
cpp_src/../nbt/tree_head/cl_bt.hpp:11:22: error: private field 'right' is not used [-Werror,-Wunused-private-field]
            Node*    right;
                     ^
3 errors generated.
make: *** [resbt] Error 1


In [3]:
Obj1 = gen_rand.Rand_set(start, stop, step)
rand_arr = Obj1.gen_rand_print(ret=1)
del(Obj1)
print("Number of sets : ", rand_arr.shape[0])

Number of sets :  998


In [4]:
with open('data/size_dup.txt', "a") as outfile:
    for i in rand_arr:
        t = np.array2string(np.array(list(i)))[1:-1]
        t = "./bin/st_bt " + t
        t = shlex.split(t)
        subprocess.run(t, stdout=outfile)

outfile.close()
del(outfile)
del(rand_arr)
del(t)
del(i)

!wc -l data/size_dup.txt;

FileNotFoundError: [Errno 2] No such file or directory: './bin/st_bt'

### Dataframe

Working with data:
*  Make dataframe(df) from 'size_dup.txt' file
*  Add one more column to df with percentage of duplicates in input
*  Print four drawings :
    * Number of inputs vs amount of duplicates
    * Number of inputs vs percentage of duplicates
    * Percentage histogram
    * Derivative of duplicates

In [ ]:
dDup = pd.read_table("data/size_dup.txt", delimiter=" ", index_col=False)
dDup['Perc'] = (dDup['Dup'] / dDup['Len']) * 1000
dDup['Perc'] = dDup['Perc'].astype(int) / 10
dDup['Der'] = dDup['Dup'].diff() # / step
dDup['d_Perc'] = dDup['Perc'].diff()
dDup.tail(5)

### Visualization 1

*  <b>Fig 1</b> : Graphic showing amount of duplicates vs length of inputs.
*  <b>Fig 2</b> : Graphic showing percentage of duplicates vs length of inputs.
*  <b>Fig 3</b> : Graphic showing how often we meet duplicates.
*  <b>Fig 4</b> : Graphic showing how changes (derivative) amount of duplicates with growing amount of inputs.

In [ ]:
dDup.plot(x='Len', y='Dup')
dDup.plot.line(x='Len', y='Perc', color='brown')
dDup.hist(column='Perc')
dDup.plot.line(x='Len', y='Der', color='black')

### Critical points

We will examine my set-to-tree algorithm for local extremas. <br>
To find local minimums we will find points in which $f'(Len)≤0$ and $f'(Len+1)>0$  <br>
To find local maximums we will find points in which $ƒ'(Len)≥0$ and $ƒ'(Len+1)<0$

In [ ]:
dDup['Der_Nxt'] = dDup['Der'].shift(-1)
dfMin = dDup[(dDup['Der'] <= 0) & (dDup['Der_Nxt'] > 0)]
dfMin = dfMin.reset_index(drop=True)
dfMin.index += 1
print("Number of local mins :", dfMin.shape[0])
dfMin['Min_Range'] = dfMin['Len'].diff()
dfMin.tail(10)

In [ ]:
dfMax = dDup[(dDup['Der'] >= 0) & (dDup['Der_Nxt'] < 0)]
dfMax = dfMax.reset_index(drop=True)
dfMax.index += 1
print("Number of local maxs :", dfMax.shape[0])
dfMax['Max_Range'] = dfMax['Len'].diff()
dfMax.tail(10)

### Min-Max frequency

We will observe case with <i>start=3 ; stop=1001; step=1</i>.<br>
<br>
How its seen, local minimum could be found : $ƒ(x)=2^{x+2}-1$ ; $x ∈ N$; <br>
Local maximum could be found : $ƒ(x)=2+3*\sum_{k=1}^{x}2^{k-1}$ ; $x ∈ N$;

In [ ]:
# Memory clean up
del(dDup)
del(dfMin)
del(dfMax)
!rm -f size_dup.txt;

### Semi-conclusion

My algorithm, converting vector to the Binary Tree, is very unoptimized. As we see, there are a lot of duplicates. Fortunately, there are no more than two same numbers in the tree, and all duplicated numbers are leaves. There will not be a problem to clean up the tree from duplicated numbers.

This algorithm does not have constant complexity, and the amount of duplicates changes linearly with increasing number of inputs. 

You can find implementation in <i>bi_tree.h</i> in <i><b>tb_node* gen_triple(int* iarr, int start, int finish);</b></i> function. Deleting of duplicates could be found in <i>bi_tree.h</i> in <i><b>void del_dup(T *root, Y *out);</b></i> function.

With next steps you will see how I work with the Binary Tree Class.

In [ ]:
print("Finish")